In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
import wrds
from scipy import stats
os.chdir('/Users/yiyujie/Desktop/program/Quantitative Asset Management')

In [108]:
conn = wrds.Connection(wrds_username='aspyyj612')

Loading library list...
Done


In [3]:
import warnings
warnings.filterwarnings('ignore')

## Q1
Using CRSP stock data, define the universe of monthly returns that can be used in calculating momentum portfolios, as well as their ranking return, following the procedure in Daniel and Moskowitz (2016) 1. Your output should be from 1927-2023

In [113]:
crsp_raw = conn.raw_sql(''' select a.permno, a.permco, a.date, b.shrcd, b.exchcd, 
                        a.ret, a.retx, a.shrout, a.prc, a.cfacshr, a.cfacpr 
                        from crspq.msf as a 
                        left join crsp.msenames as b 
                        on a.permno=b.permno 
                        and b.namedt<=a.date 
                        and a.date<=b.nameendt 
                        where a.date between '01/01/1924'and'02/28/2024' 
                        ''')
crsp_raw = crsp_raw.sort_values(['permno','date']).reset_index(drop=True).copy()
crsp_raw[['permno','permco']] = crsp_raw[['permno','permco']].astype(int)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'], format='%Y-%m-%d', errors='ignore')

dlret_raw = conn.raw_sql('''select permno, dlret, dlstdt, dlstcd 
                            from crspq.msedelist
                         ''')
dlret_raw = dlret_raw.sort_values(['permno','dlstdt']).reset_index(drop=True).copy()
dlret_raw.permno = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt']) 
dlret_raw['date'] = dlret_raw['dlstdt'] + MonthEnd(0)

CRSP_stocks = crsp_raw.merge(dlret_raw, how='outer', on=['date','permno'])[['permno','date','shrcd','exchcd','dlret','prc','ret','shrout']]
CRSP_stocks.columns = ['PERMNO', 'date', 'SHRCD', 'EXCHCD', 'DLRET', 'PRC','RET', 'SHROUT']

In [114]:
CRSP_stocks_1 = CRSP_stocks.copy()

In [102]:
CRSP_stocks = CRSP_stocks_1.copy()

In [123]:
CRSP_stocks

,PERMNO,date,SHRCD,EXCHCD,DLRET,PRC,RET,SHROUT
0,10006,1925-12-31,10.0,1.0,NaN,109.0,NaN,600.0
1,10014,1925-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2,10022,1925-12-31,10.0,1.0,NaN,56.0,NaN,200.0
3,10030,1925-12-31,10.0,1.0,NaN,150.0,NaN,156.0
4,10049,1925-12-31,12.0,1.0,NaN,74.0,NaN,250.0
...,...,...,...,...,...,...,...,...
5054878,93426,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN
5054879,93427,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN
5054880,93429,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN
5054881,93434,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
def PS3_Q1(CRSP_stocks):
    # CRSP Sharecode 10 or 11, CRSP exchange code of 1, 2 or 3 (NYSE, AMEX or NASDAQ)
    CRSP_stocks = CRSP_stocks[(CRSP_stocks['SHRCD'].isin([10, 11])) & 
            (CRSP_stocks['EXCHCD'].isin([1, 2, 3]))]
    CRSP_stocks['Year'] = CRSP_stocks['date'].dt.year
    CRSP_stocks['Month'] = CRSP_stocks['date'].dt.month
    
    # calculate Market value
    CRSP_stocks['Mkt_Cap'] = CRSP_stocks['PRC'].abs() * CRSP_stocks['SHROUT'] * 1e-6
    CRSP_stocks['lag_Mkt_Cap'] = CRSP_stocks[["PERMNO", "Mkt_Cap"]].groupby("PERMNO").shift(1)
    
    # calculate RET
    CRSP_stocks['DLRET'] = pd.to_numeric(CRSP_stocks['DLRET'], errors='coerce')
    CRSP_stocks['RET'] = pd.to_numeric(CRSP_stocks['RET'], errors='coerce')
    # CRSP_stocks = CRSP_stocks.loc[CRSP_stocks['DLRET'].notna()|CRSP_stocks['RET'].notna()].reset_index(drop=True)
    CRSP_stocks['RET'] = np.where(CRSP_stocks['RET'].notna() & CRSP_stocks['DLRET'].notna(), 
                            (1 + CRSP_stocks['RET']) * (1 + CRSP_stocks['DLRET']) - 1, CRSP_stocks['RET'])
    CRSP_stocks['RET'] = np.where(CRSP_stocks['RET'].isna()  & CRSP_stocks['DLRET'].notna(), CRSP_stocks['DLRET'], CRSP_stocks['RET'])

    # must have return for t-2 and me for t-1 and price for t-13
    CRSP_stocks = CRSP_stocks[CRSP_stocks.groupby('PERMNO')['PRC'].shift(13).notna()]
    CRSP_stocks = CRSP_stocks[CRSP_stocks.groupby('PERMNO')['RET'].shift(2).notna()]
    CRSP_stocks = CRSP_stocks[CRSP_stocks['lag_Mkt_Cap'].notna()]
    # CRSP_stocks = CRSP_stocks[CRSP_stocks['lag_Mkt_Cap'] > 0]
    
    # calculate cumulative return from month t-12 through month t-2
    CRSP_stocks.loc[:,'log_ret'] = np.log(1 + CRSP_stocks['RET'] + 1e-7)
    CRSP_stocks['Ranking_Ret'] = CRSP_stocks.groupby('PERMNO')['log_ret'].transform(lambda x: x.rolling(window=11, min_periods=1).sum().shift(2))
    CRSP_stocks['Ranking_Ret'] = np.exp(CRSP_stocks['Ranking_Ret']) - 1
    
    CRSP_stocks = CRSP_stocks[CRSP_stocks['Ranking_Ret'].notna()]
    
    return CRSP_stocks[CRSP_stocks['Year']>=1927].rename(columns={'RET':'Ret'})[['Year', 'Month', 'PERMNO', 'EXCHCD', 'lag_Mkt_Cap', 'Ret', 'Ranking_Ret']].reset_index(drop=True)

CRSP_Stocks_Momentum = PS3_Q1(CRSP_stocks)

In [122]:
CRSP_Stocks_Momentum

,Year,Month,PERMNO,EXCHCD,lag_Mkt_Cap,Ret,Ranking_Ret,DM_decile,NYSE_decile
0,1927,5,10006,1.0,0.060450,0.078164,-0.028103,5,5.0
1,1927,5,10022,1.0,0.009894,0.012887,0.090234,10,10.0
2,1927,5,10030,1.0,0.022308,0.076923,-0.013793,6,6.0
3,1927,5,10057,1.0,0.002750,0.181818,0.184211,10,10.0
4,1927,5,10073,1.0,0.001690,-0.040816,-0.083333,2,2.0
...,...,...,...,...,...,...,...,...,...
3293101,2023,12,93397,3.0,0.273509,0.362171,0.136160,9,NaN
3293102,2023,12,93423,1.0,2.080071,0.007229,-0.173931,5,4.0
3293103,2023,12,93426,1.0,0.381430,0.117416,-0.262508,4,3.0
3293104,2023,12,93434,3.0,0.028277,0.065449,-0.396668,3,NaN


In [94]:
#stocks2['aux'] = 
np.where(CRSP_stocks['RET'] < -.99,-.99,CRSP_stocks['RET'])


array([nan, nan, nan, ..., nan, nan, nan])

## Q2
Define the monthly momentum portfolio decile of each stock as defined by both Daniel and Moskowitz (2016) and Kenneth R. French. Your output should be from 1927-2023.

In [117]:
def PS3_Q2(CRSP_Stocks_Momentum):
    # equal number of firms in each portfolio
    CRSP_Stocks_Momentum['DM_decile'] = CRSP_Stocks_Momentum.groupby(['Year','Month'])['Ranking_Ret'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
    
    # each of the portfolios has an equal number of NYSE firms
    CRSP_Stocks_Momentum.loc[CRSP_Stocks_Momentum.EXCHCD==1,'NYSE_decile'] = CRSP_Stocks_Momentum[CRSP_Stocks_Momentum.EXCHCD==1].groupby(['Year','Month'])['Ranking_Ret'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
    breakpoints = CRSP_Stocks_Momentum.groupby(['Year','Month','NYSE_decile'])['Ranking_Ret'].max().groupby(['Year','Month']).apply(lambda x: [-np.inf] + x.tolist() + [np.inf]).reset_index().rename(columns={'Ranking_Ret':'breakpoints'})
    CRSP_Stocks_Momentum = pd.merge(breakpoints, CRSP_Stocks_Momentum, on=['Year','Month'])
    def find(ret,breakpoint):
        for i in range(len(breakpoint)):
            if ret <= breakpoint[i]:
                return i
    CRSP_Stocks_Momentum['KRF_decile'] = CRSP_Stocks_Momentum.apply(lambda x : find(x['Ranking_Ret'],x['breakpoints']),axis=1)
    return CRSP_Stocks_Momentum[['Year', 'Month', 'PERMNO', 'lag_Mkt_Cap', 'Ret', 'DM_decile', 'KRF_decile', 'EXCHCD']]

CRSP_Stocks_Momentum_decile = PS3_Q2(CRSP_Stocks_Momentum)

In [80]:
def PS3_Q2(CRSP_Stocks_Momentum):
    # equal number of firms in each portfolio
    CRSP_Stocks_Momentum['DM_decile'] = CRSP_Stocks_Momentum.groupby(['Year','Month'])['Ranking_Ret'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
    
    # each of the portfolios has an equal number of NYSE firms
    def cal_krf_deciles(df):
        nyse = df[df['EXCHCD'] == 1]
        nyse['NYSE_decile']= nyse['Ranking_Ret'].rank(method='first', pct=True)
        breakpoints = nyse['NYSE_decile'].quantile(np.arange(0.1,1.0,0.1)).values
        print(breakpoints)
        df['KRF_decile'] = pd.cut(df['Ranking_Ret'], bins = [-np.inf]+ list(breakpoints) + [np.inf],labels = range(1,11))
        return df

    CRSP_Stocks_Momentum = CRSP_Stocks_Momentum.groupby(['Year','Month']).apply(cal_krf_deciles).reset_index(level = [0, 1], drop=True)
    return CRSP_Stocks_Momentum[['Year', 'Month', 'PERMNO', 'lag_Mkt_Cap', 'Ret', 'DM_decile', 'KRF_decile', 'EXCHCD']]

CRSP_Stocks_Momentum_decile = PS3_Q2(CRSP_Stocks_Momentum)

[nan nan nan nan nan nan nan nan nan]


ValueError: bins must increase monotonically.

In [121]:
CRSP_Stocks_Momentum_decile

,Year,Month,PERMNO,lag_Mkt_Cap,Ret,DM_decile,KRF_decile,EXCHCD
0,1927,5,10006,0.060450,0.078164,5,5,1.0
1,1927,5,10022,0.009894,0.012887,10,10,1.0
2,1927,5,10030,0.022308,0.076923,6,6,1.0
3,1927,5,10057,0.002750,0.181818,10,10,1.0
4,1927,5,10073,0.001690,-0.040816,2,2,1.0
...,...,...,...,...,...,...,...,...
3293101,2023,12,93397,0.273509,0.362171,9,8,3.0
3293102,2023,12,93423,2.080071,0.007229,5,4,1.0
3293103,2023,12,93426,0.381430,0.117416,4,3,1.0
3293104,2023,12,93434,0.028277,0.065449,3,2,3.0


## Q3
Calculate the monthly momentum portfolio decile returns as defined by both Daniel and Moskowitz (2016) and Kenneth R. French. Your output should be from 1927-2023.

In [20]:
import pandas_datareader
FF_mkt = pandas_datareader.famafrench.FamaFrenchReader('F-F_Research_Data_Factors',start='1926-07',end='2024-02').read()[0]/100
FF_mkt = FF_mkt.reset_index()
FF_mkt['Year'] = FF_mkt['Date'].astype('str').apply(lambda x: int(x[:4]))
FF_mkt['Month'] = FF_mkt['Date'].astype('str').apply(lambda x: int(x[5:]))
FF_mkt[['Mkt-RF','SMB','HML','RF']] = FF_mkt[['Mkt-RF','SMB','HML','RF']].astype(float)
FF_mkt

,Date,Mkt-RF,SMB,HML,RF,Year,Month
0,1926-07,0.0296,-0.0256,-0.0243,0.0022,1926,7
1,1926-08,0.0264,-0.0117,0.0382,0.0025,1926,8
2,1926-09,0.0036,-0.0140,0.0013,0.0023,1926,9
3,1926-10,-0.0324,-0.0009,0.0070,0.0032,1926,10
4,1926-11,0.0253,-0.0010,-0.0051,0.0031,1926,11
...,...,...,...,...,...,...,...
1167,2023-10,-0.0319,-0.0388,0.0018,0.0047,2023,10
1168,2023-11,0.0884,-0.0002,0.0164,0.0044,2023,11
1169,2023-12,0.0487,0.0634,0.0493,0.0043,2023,12
1170,2024-01,0.0071,-0.0509,-0.0238,0.0047,2024,1


In [118]:
def PS3_Q3(CRSP_Stocks_Momentum_decile, FF_mkt):
    # CRSP_Stocks_Momentum_decile = CRSP_Stocks_Momentum_decile[CRSP_Stocks_Momentum_decile['lag_Mkt_Cap'] > 0]
    
    CRSP_Stocks_Momentum_decile = CRSP_Stocks_Momentum_decile.merge(CRSP_Stocks_Momentum_decile[['Year','Month','DM_decile','lag_Mkt_Cap']].groupby(['Year','Month','DM_decile']).sum()\
                        .reset_index().rename(columns={"lag_Mkt_Cap": "LmeTotal"}),on=['Year','Month','DM_decile'],how='left')
    CRSP_Stocks_Momentum_decile['DM_Ret'] = CRSP_Stocks_Momentum_decile['Ret'] * CRSP_Stocks_Momentum_decile['lag_Mkt_Cap'] / CRSP_Stocks_Momentum_decile['LmeTotal']
    CRSP_Stocks_Momentum_returns = CRSP_Stocks_Momentum_decile.groupby(['Year','Month','DM_decile'])['DM_Ret'].sum().reset_index()

    CRSP_Stocks_Momentum_decile = CRSP_Stocks_Momentum_decile.merge(CRSP_Stocks_Momentum_decile[['Year','Month','KRF_decile','lag_Mkt_Cap']].groupby(['Year','Month','KRF_decile']).sum()\
                        .reset_index().rename(columns={"lag_Mkt_Cap":"LmeTotal_KRF"}),on=['Year','Month','KRF_decile'],how='left')
    CRSP_Stocks_Momentum_decile['KRF_Ret'] = CRSP_Stocks_Momentum_decile['Ret'] * CRSP_Stocks_Momentum_decile['lag_Mkt_Cap'] / CRSP_Stocks_Momentum_decile['LmeTotal_KRF']
    CRSP_Stocks_Momentum_returns = pd.merge(CRSP_Stocks_Momentum_returns, CRSP_Stocks_Momentum_decile.groupby(['Year','Month','KRF_decile'])['KRF_Ret']
                    .sum().reset_index(), left_on=['Year', 'Month','DM_decile'], right_on = ['Year', 'Month','KRF_decile'])
    
    CRSP_Stocks_Momentum_returns = pd.merge(CRSP_Stocks_Momentum_returns,FF_mkt[['Year', 'Month','RF']], on = ['Year', 'Month'])
    CRSP_Stocks_Momentum_returns = CRSP_Stocks_Momentum_returns.rename(columns={'DM_decile':'decile'})
    return CRSP_Stocks_Momentum_returns[['Year','Month','decile','DM_Ret','KRF_Ret','RF']]

CRSP_Stocks_Momentum_returns = PS3_Q3(CRSP_Stocks_Momentum_decile, FF_mkt)

In [125]:
CRSP_Stocks_Momentum_returns.iloc[:,:-2]

,Year,Month,decile,DM_Ret,KRF_Ret
0,1927,5,1,0.051603,0.051603
1,1927,5,2,0.051443,0.051443
2,1927,5,3,0.038674,0.038674
3,1927,5,4,0.068007,0.068007
4,1927,5,5,0.055889,0.055889
...,...,...,...,...,...
11595,2023,12,6,0.059351,0.054378
11596,2023,12,7,0.046388,0.046997
11597,2023,12,8,0.042494,0.030986
11598,2023,12,9,0.043643,0.044107


## Q4
Recreate Table 1 from Daniel and Moskowitz (2016), but exclude the rows for α, t(α), β,
and sk(d), as well as the Market column. Ensure your version closely follows the original table’s format and methods. Use the entire data sample for your statistics.

Additionally, calculate and include a new row at the bottom of your table showing the correlation between your portfolio returns and the decile-based breakpoints from Daniel and Moskowitz (2016), as well as the returns listed on Daniel’s website.2 Refer to the extra materials provided at the end of this assignment for an example on the table format and expected outcome

In [23]:
with open('m_m_pt_tot.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
data = []
for line in lines:
    data.append(line.strip().split())
DM_returns = pd.DataFrame(data,columns=['date','decile','DM_Ret','me','unkown'])
DM_returns['date'] = pd.to_datetime(DM_returns['date'])
DM_returns['Year'] = DM_returns['date'].dt.year
DM_returns['Month'] = DM_returns['date'].dt.month
DM_returns['decile'] = pd.to_numeric(DM_returns['decile'], errors='coerce')
DM_returns['DM_Ret'] = pd.to_numeric(DM_returns['DM_Ret'], errors='coerce')
DM_returns = DM_returns[['Year','Month','decile','DM_Ret']]
DM_returns

,Year,Month,decile,DM_Ret
0,1927,1,1,-0.03362
1,1927,1,2,-0.04584
2,1927,1,3,0.02755
3,1927,1,4,-0.00319
4,1927,1,5,-0.00294
...,...,...,...,...
10795,2016,12,6,0.02198
10796,2016,12,7,0.02699
10797,2016,12,8,0.02151
10798,2016,12,9,0.00435


In [119]:
def PS3_Q4(CRSP_Stocks_Momentum_returns, DM_returns):
    result = pd.DataFrame()

    CRSP_Stocks_Momentum_returns['excess_ret'] = CRSP_Stocks_Momentum_returns['DM_Ret'] - CRSP_Stocks_Momentum_returns['RF']
    CRSP_Stocks_Momentum_returns = pd.merge(CRSP_Stocks_Momentum_returns, DM_returns.rename(columns={'DM_Ret':'DM_Ret_1'}), on = ['Year', 'Month', 'decile'])
    for decile in range(1, 11):
        decile_df = CRSP_Stocks_Momentum_returns[CRSP_Stocks_Momentum_returns['decile'] == decile]
        excess_ret = decile_df['excess_ret'].mean() * 100 * 12
        std_dev = decile_df['DM_Ret'].std() * 100 * np.sqrt(12)
        sharp_ratio = excess_ret / std_dev if std_dev != 0 else np.nan
        skewness = decile_df['DM_Ret'].skew()
        correlation = decile_df['DM_Ret'].corr(decile_df['DM_Ret_1'])
        
        result.at['Excess Return', f'Decile {decile}'] = excess_ret
        result.at['Volatility', f'Decile {decile}'] = std_dev
        result.at['Sharp Ratio', f'Decile {decile}'] = sharp_ratio
        result.at['Skewness', f'Decile {decile}'] = skewness
        result.at['Correlation', f'Decile {decile}'] = correlation
    
    # calculate Winner-Minus-Loser WML which is 10 - 1
    wml_df = pd.merge(CRSP_Stocks_Momentum_returns.loc[CRSP_Stocks_Momentum_returns['decile']==10], CRSP_Stocks_Momentum_returns.loc[CRSP_Stocks_Momentum_returns['decile']==1], on=['Year','Month'])
    wml_df['WML'] = wml_df['DM_Ret_x'] - wml_df['DM_Ret_y']
    result.at['Excess Return', 'WML'] = wml_df['WML'].mean() * 100 * 12
    result.at['Volatility', 'WML'] = wml_df['WML'].std() * 100 * np.sqrt(12)
    result.at['Sharp Ratio', 'WML'] = result.at['Excess Return', 'WML'] / result.at['Volatility', 'WML'] if result.at['Volatility', 'WML'] != 0 else np.nan
    result.at['Skewness', 'WML'] = wml_df['WML'].skew()
    wml_df['WML_1'] = wml_df['DM_Ret_1_x'] - wml_df['DM_Ret_1_y']
    result.at['Correlation', 'WML'] = wml_df['WML'].corr(wml_df['WML_1'])
    
    return result

PS3_Q4(CRSP_Stocks_Momentum_returns, DM_returns)

,Decile 1,Decile 2,Decile 3,Decile 4,Decile 5,Decile 6,Decile 7,Decile 8,Decile 9,Decile 10,WML
Excess Return,-0.484850,2.541528,3.870449,6.936221,7.260721,7.561204,9.245716,10.401829,11.474769,14.722097,15.206947
Volatility,36.109912,29.204288,25.034717,22.695763,20.773654,20.030431,18.782787,18.734040,19.885195,23.403896,29.567765
Sharp Ratio,-0.013427,0.087026,0.154603,0.305617,0.349516,0.377486,0.492244,0.555237,0.577051,0.629045,0.514308
Skewness,1.735662,1.424363,1.090587,1.343999,0.917714,0.813077,0.237355,-0.074866,-0.305293,-0.395108,-2.281976
Correlation,0.992961,0.990059,0.990942,0.991186,0.990956,0.991697,0.993143,0.994348,0.993624,0.995267,0.986094


## Q5
Recreate Table 1 from Daniel and Moskowitz (2016), but exclude the rows for α, t(α), β,
and sk(d), as well as the Market column. Ensure your version closely follows the original table’s format and methods. Use the entire data sample for your statistics.

Importantly, use NYSE breakpoint instead (i.e., Ken French portfolio breakpoints).

As in the previous question, calculate and include a new row at the bottom of your table showing the correlation between your portfolio returns and the decile-based breakpoints from Daniel and Moskowitz (2016), as well as the returns listed on Daniel’s website. Refer to the extra materials provided at the end of this assignment for an example on the table format and expected outc

In [25]:
KRF_Returns = pandas_datareader.famafrench.FamaFrenchReader('10_Portfolios_Prior_12_2',start='1926-07',end='2024-02').read()[0]/100
KRF_Returns = pd.melt(KRF_Returns.reset_index(), id_vars=['Date'], var_name='decile', value_name='KRF_Ret')
decile_mapping = {
    'Lo PRIOR': 1,
    'PRIOR 2': 2,
    'PRIOR 3': 3,
    'PRIOR 4': 4,
    'PRIOR 5': 5,
    'PRIOR 6': 6,
    'PRIOR 7': 7,
    'PRIOR 8': 8,
    'PRIOR 9': 9,
    'Hi PRIOR': 10,
}
KRF_Returns['decile'] = KRF_Returns['decile'].map(decile_mapping)
KRF_Returns['Year'] = KRF_Returns['Date'].dt.year
KRF_Returns['Month'] = KRF_Returns['Date'].dt.month
KRF_Returns = KRF_Returns[['Year','Month','decile','KRF_Ret']]
KRF_Returns

,Year,Month,decile,KRF_Ret
0,1927,1,1,-0.0332
1,1927,2,1,0.0753
2,1927,3,1,-0.0323
3,1927,4,1,0.0204
4,1927,5,1,0.0272
...,...,...,...,...
11655,2023,10,10,-0.0409
11656,2023,11,10,0.1283
11657,2023,12,10,0.0391
11658,2024,1,10,0.0514


In [124]:
def PS3_Q5(CRSP_Stocks_Momentum_returns, KRF_returns):
    result = pd.DataFrame()

    CRSP_Stocks_Momentum_returns['excess_ret'] = CRSP_Stocks_Momentum_returns['KRF_Ret'] - CRSP_Stocks_Momentum_returns['RF']
    CRSP_Stocks_Momentum_returns = pd.merge(CRSP_Stocks_Momentum_returns, KRF_returns.rename(columns={'KRF_Ret':'KRF_Ret_1'}), on = ['Year', 'Month', 'decile'])
    for decile in range(1, 11):
        decile_df = CRSP_Stocks_Momentum_returns[CRSP_Stocks_Momentum_returns['decile'] == decile]
        excess_ret = decile_df['excess_ret'].mean() * 100 * 12
        std_dev = decile_df['KRF_Ret'].std() * 100 * np.sqrt(12)
        sharp_ratio = excess_ret / std_dev if std_dev != 0 else np.nan
        skewness = decile_df['KRF_Ret'].skew()
        correlation = decile_df['KRF_Ret'].corr(decile_df['KRF_Ret_1'])
        
        result.at['Excess Return', f'Decile {decile}'] = excess_ret
        result.at['Volatility', f'Decile {decile}'] = std_dev
        result.at['Sharp Ratio', f'Decile {decile}'] = sharp_ratio
        result.at['Skewness', f'Decile {decile}'] = skewness
        result.at['Correlation', f'Decile {decile}'] = correlation
    
    # calculate Winner-Minus-Loser WML which is 10 - 1
    wml_df = pd.merge(CRSP_Stocks_Momentum_returns.loc[CRSP_Stocks_Momentum_returns['decile']==10], CRSP_Stocks_Momentum_returns.loc[CRSP_Stocks_Momentum_returns['decile']==1], on=['Year','Month'])
    wml_df['WML'] = wml_df['KRF_Ret_x'] - wml_df['KRF_Ret_y']
    result.at['Excess Return', 'WML'] = wml_df['WML'].mean() * 100 * 12
    result.at['Volatility', 'WML'] = wml_df['WML'].std() * 100 * np.sqrt(12)
    result.at['Sharp Ratio', 'WML'] = result.at['Excess Return', 'WML'] / result.at['Volatility', 'WML'] if result.at['Volatility', 'WML'] != 0 else np.nan
    result.at['Skewness', 'WML'] = wml_df['WML'].skew()
    wml_df['WML_1'] = wml_df['KRF_Ret_1_x'] - wml_df['KRF_Ret_1_y']
    result.at['Correlation', 'WML'] = wml_df['WML'].corr(wml_df['WML_1'])
    
    return result

PS3_Q5(CRSP_Stocks_Momentum_returns, KRF_Returns)

,Decile 1,Decile 2,Decile 3,Decile 4,Decile 5,Decile 6,Decile 7,Decile 8,Decile 9,Decile 10,WML
Excess Return,1.744846,5.615533,6.193213,7.662488,7.679034,8.198007,8.947554,10.359154,10.795476,14.218736,12.473890
Volatility,34.132315,27.857545,23.549500,21.860170,20.140640,19.852506,18.496169,18.340458,19.105284,22.178861,27.281737
Sharp Ratio,0.051120,0.201580,0.262987,0.350523,0.381271,0.412946,0.483752,0.564825,0.565052,0.641094,0.457225
Skewness,1.846869,1.508396,1.209932,1.352186,0.931990,0.705749,0.198133,-0.017797,-0.323445,-0.516943,-2.505119
Correlation,0.995853,0.993730,0.993066,0.993402,0.992284,0.993451,0.991575,0.994050,0.994434,0.996258,0.990571


## Q6
Has the momentum anomaly worked in the past few years? Show and discuss empirical evidence.

From the data tables above, which presumably represent metrics for different deciles of stocks sorted by past returns (from low to high momentum), there are several key observations:

Excess Return: Decile 10, which represents stocks with the highest past momentum, consistently shows higher excess returns compared to Decile 1, indicating that stocks with higher past returns continue to perform better.
Volatility: Higher momentum stocks (Decile 10) tend to have higher volatility compared to lower momentum stocks (Decile 1). This suggests higher risks associated with high momentum stocks.
Sharpe Ratio: Generally, the Sharpe Ratios improve as we move from Decile 1 to Decile 10, which suggests that the additional risk of high-momentum stocks is compensated by higher returns. This ratio is crucial as it adjusts the returns of an investment by the risk taken to achieve those returns.
Skewness: This varies across deciles. Negative skewness in the higher deciles could indicate the potential for extreme negative returns.
Correlation: High correlation figures across all deciles suggest that momentum stocks may move together, which could pose a diversification challenge.

The data shows that the momentum anomaly has been effective, with high momentum stocks yielding higher excess returns despite their higher risk profiles. This trend is consistent across different years, as evidenced by the provided data.

## Q7
Would you implement this trading strategy if you were running your own fund? What are
the main implementation challenges to consider?

Considerations for Implementation:

Risk vs. Reward: The strategy involves higher risk (as indicated by higher volatility and sometimes negative skewness), which needs to be balanced against the potential for higher returns. Proper risk management strategies must be employed.
Market Impact and Liquidity: Implementing momentum strategies on a large scale can impact market prices, especially when buying or selling large positions in high-momentum stocks. Liquidity constraints and the potential impact on market prices must be considered.
Transaction Costs: Frequent trading to maintain a momentum portfolio can incur significant transaction costs, which can erode profits.
Behavioral Factors: Momentum strategies can be susceptible to market sentiment and investor behaviors that can lead to rapid reversals. Understanding market psychology and behavioral finance is crucial in managing these strategies.
Capacity of Strategy: The effectiveness of momentum strategies might degrade if too much capital chases the same signals. The capacity to absorb new capital without diminishing returns is a critical consideration.
Personal Decision:
If I were running my own fund, the decision to implement a momentum trading strategy would depend on:

My risk tolerance and the risk profile of my clients.
My capacity to manage the high volatility and potential drawdowns associated with such a strategy.
My operational capability to manage the high-frequency trading and associated costs effectively.
Overall, while the momentum anomaly has demonstrated potential for higher returns, it comes with significant risks and operational challenges that need careful management. The decision to implement such a strategy should be based on a thorough understanding of these dynamics and a robust risk management framework.